Describe the purpose of this script quick and easy.

In [1]:
## Load libraries

import pandas as pd
import numpy as np
import os
from sklearn.metrics import cohen_kappa_score
from itertools import combinations
from pathlib import Path


## 1) Experiments Nr. 1 (2025-12-09)

## 1.1) Show Prompt Template (Nr. 1): 

In [2]:
# Load prompt template nr.1


# Define function to load templates
def load_prompt_template(template_name: str) -> str:
    """Load a prompt template from the LLMs_prompt_templates directory."""
    prompt_path = Path("LLMs_prompt_templates") / f"{template_name}.md"

    if not prompt_path.exists():
        raise FileNotFoundError(f"Prompt template not found: {prompt_path}")
    
    with open(prompt_path, "r", encoding = "utf-8") as file:
        return file.read()
    


# Use function
prompt_1 = load_prompt_template("prompt_template_1")
print(prompt_1)



Du bist ein hilfreicher Assistent, der DeLFI Publikationen (DeLFI - e-Learning Fachtagung Informatik)  im Rahmen einer quantitativen Inhaltsanalyse labelt. Die DELFI-Tagungen beschäftigen sich mit allen Informatik-Aspekten internet-, medien- und rechnergestützter Lehr- und Lernformen in Schule, Hochschule, beruflicher und privater Aus- und Weiterbildung. Deine Aufgabe ist es, die angehängte Publikation (“scratchdrone.pdf”) auf Basis des folgenden Schemas zu labeln: 

In einem ersten Schritt, sollen folgende Variablen gelabelt werden: 

1) Wurde eine Evaluation beschrieben? (ja = 1, nein = 0) 
2) Wird eine einzelne Software/Medium/Tool diskutiert (ja = 1, nein = 0) 
3) Geht es um ein konkretes Lehr-Lern-Setting? (ja = 1, nein = 0)

In einem zweiten Schritt soll der Evaluationsfokus einer Publikation anhand von vier Paradigmen gelabelt werden: 

4) Prozess orientiertes Paradigma (Effizienz): Wird die Effizienz betrachtet, die das Medium auf einen Prozess zu dessen Verbesserung ausübt? (-

## 1.2) Show LLM Configs (Nr. 1): 

In [3]:
# Load LLM Configs


# Define function to load templates
def load_LLM_config(config_name: str) -> str:
    """Load a config file from the LLMs_configurations directory."""
    config_path = Path("LLMs_configurations") / f"{config_name}.md"

    if not config_path.exists():
        raise FileNotFoundError(f"Config file not found: {config_path}")
    
    with open(config_path, "r", encoding = "utf-8") as file:
        return file.read()
    

# Use function
LLM_config = load_LLM_config("LLM_configs_1")
print(LLM_config)

# LLM Configuration Summary

This document summarizes the configuration details for various large language models used in the preliminary experimens.

First experiments were done with "prompt_template_1.md" on: 2025-12-09


- **LLM:** Claude Sonnet 4.5
    - **Access:** web_interface  
    - **Style:** Concise  
    - **Tools:** No tools use  
        - e.g., No Web Search, No Extended Thinking


- **LLM:** ChatGPT
    - **Access:** web_interface  
    - **Personality:** default  
    - **Tools:** No tools use  
        - e.g., Thinking, DeepResearch, Study and learn, web search


- **LLM:** Gemini 2.5 (Flash-Lite Preview 09-2025)
    - **Access:** web_interface  
    - **Tools:** No tools use 
    - **Thinking budget:** off  
    - **Temperature:** 0  
    - **Top-p:** 0.95

- **LLM:** qwen/qwen3-30b-a3b-2507 (rag-v1)
    - **Access:** LM Studio App (locally)  
    - **No tools use**
    


## 1.3) Load Data 

### 1.3.1) Data from [Reference Paper](https://dl.gi.de/server/api/core/bitstreams/55b17878-2f79-40ce-b574-313db7cfaf11/content): Paper 2014-2016

In [4]:
## 1) Load excel file with 28 papers from server 

# Define the base mount point for each OS
if Path("/Volumes").exists(): #MacOs
    base_path = Path("/Volumes")
elif Path("/mnt").exists(): #Linux
    base_path = Path("/mnt")
elif Path("Z:/").exists(): #Windows
    base_path = Path("Z:")
else:
    raise FileNotFoundError("Network drive not found. Please mount if first")

# Build the full path using pathlib (automatically uses correct separators)
file_path = base_path / "Förderprojekte" / "24-0010_KTS_RSE-Award" /"04_Case_Studies" /"delfi" /"reference_paper" /"evaluation"/ "Evaluationen3.xlsx"

# Check if file exists
if not file_path.exists():
    raise FileNotFoundError(f"File not found at {file_path}")

# Load data 
df = pd.read_excel(file_path)
print(len(df))
df.head()

83


,DelFI Band,Beitragstitelseite,Evaluation durchgeführt,Einzelne Software,Lehr-Lernsetting,Fokus Medialität,Fokus Akzeptanz,Fokus Normalität der Nutzung,Fokus Innovation,Fokus Empirischer Lernerfolg,Softwareklasse,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,X-Koordinate,Y-Koordinate
0,2005,23,0,0,1,-1,-1,0,1,-1,1,NaN,NaN,NaN,NaN,NaN,NaN
1,2005,57,0,0,1,-1,-1,0,-1,-1,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2005,69,0,0,1,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2005,81,1,0,1,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2005,93,0,1,1,-1,1,1,-1,-1,0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
## 2) Filter papers (passed the first layer of evaluation: Evaluation durchgeführt, Einzelne Software, Lehr-Lernsetting), should be n = 28

df_subset = df.loc[(df["Evaluation durchgeführt"] == 1) & (df["Einzelne Software"] == 1) & (df["Lehr-Lernsetting"] == 1) & (df["DelFI Band"] >= 2014)]
print(len(df_subset)) # n = 28
df_subset 

28


,DelFI Band,Beitragstitelseite,Evaluation durchgeführt,Einzelne Software,Lehr-Lernsetting,Fokus Medialität,Fokus Akzeptanz,Fokus Normalität der Nutzung,Fokus Innovation,Fokus Empirischer Lernerfolg,Softwareklasse,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,X-Koordinate,Y-Koordinate
12,2014,109,1,1,1,0,0,-1,1,-1,0,NaN,NaN,NaN,NaN,1.0,-1.0
15,2014,133,1,1,1,-1,0,1,-1,-1,0,NaN,NaN,NaN,NaN,0.0,0.0
16,2014,145,1,1,1,1,-1,-1,-1,-1,0,NaN,NaN,NaN,NaN,0.0,1.0
22,2014,193,1,1,1,-1,1,-1,-1,1,0,NaN,NaN,NaN,NaN,-1.0,0.0
24,2014,217,1,1,1,-1,1,1,-1,-1,0,NaN,NaN,NaN,NaN,0.0,0.0
28,2014,259,1,1,1,0,-1,-1,-1,1,0,NaN,NaN,NaN,NaN,-1.0,0.5
33,2014,289,1,1,1,-1,1,-1,0,-1,0,NaN,NaN,NaN,NaN,0.5,-0.5
36,2014,301,1,1,1,-1,1,-1,-1,0,0,NaN,NaN,NaN,NaN,-0.5,0.0
42,2015,81,1,1,1,-1,1,-1,0,1,0,NaN,NaN,NaN,NaN,-1.0,-0.5
44,2015,107,1,1,1,-1,-1,-1,1,1,0,NaN,NaN,NaN,NaN,0.0,-1.0


In [6]:
### Get the distribution of paradigms within papers from 2014-2016

## 2) Show all papers with focus medialität (Fokus Medialität = 1) from 2014-2016

df_subset_medialität = df_subset.loc[df_subset["Fokus Medialität"] == 1]
print(f'Paper mit Fokus Medialität: {len(df_subset_medialität)}')


## 3) Show all papers with focus societal aceptance (Fokus Akzeptanz = 1) from 2014-2016

df_subset_akzeptanz = df_subset.loc[df_subset["Fokus Akzeptanz"] == 1]
print(f'Paper mit Fokus Akzeptanz: {len(df_subset_akzeptanz)}')

## 4) Show all papers with focus normality (Fokus Normalität der Nutzung = 1) from 2014-2016

df_subset_akzeptanz = df_subset.loc[df_subset["Fokus Normalität der Nutzung"] == 1]
print(f'Paper mit Fokus Normalität der Nutzung: {len(df_subset_akzeptanz)}')

## 5) Show all papers with innovation focus (Fokus Innovation = 1) from 2014-2016

df_subset_innovation = df_subset.loc[(df_subset["Fokus Innovation"] == 1) ]
print(f'Paper mit Fokus Innovation: {len(df_subset_innovation)}')

## 6) Show all papers with evaluation focus (Fokus Empirischer Lernerfolg = 1) from 2014-2016
df_subset_lernerfolg = df_subset.loc[(df["Fokus Empirischer Lernerfolg"] == 1)]
print(f'Paper mit Fokus Empirischer Lernerfolg: {len(df_subset_lernerfolg)}')

## 7) Show all papers with innovation & evaluation focus from 2014-2016
df_subset_innovation_lernerfolg = df_subset.loc[(df["Fokus Innovation"] == 1) & (df_subset["Fokus Empirischer Lernerfolg"] == 1)]
print(f'Paper mit Fokus Innovation & Empirischer Lernerfolg: {len(df_subset_innovation_lernerfolg)}')


Paper mit Fokus Medialität: 5
Paper mit Fokus Akzeptanz: 19
Paper mit Fokus Normalität der Nutzung: 4
Paper mit Fokus Innovation: 4
Paper mit Fokus Empirischer Lernerfolg: 10
Paper mit Fokus Innovation & Empirischer Lernerfolg: 2


#### 1.3.1.1) Innovation Paper

In [7]:
## 8) Sampling idea for labeling: focus on innovation (Innovation) & evaluation (Empirischer Lernerfolg)

# papers before 2017:

# 1 paper from innovation
# 1 paper from evaluation 
# 1 or 2 papers from innovation & evaluation (not part of the first 2)


# papers after 2017: 

# 2 papers (classified by LLMs)

df_subset_innovation

,DelFI Band,Beitragstitelseite,Evaluation durchgeführt,Einzelne Software,Lehr-Lernsetting,Fokus Medialität,Fokus Akzeptanz,Fokus Normalität der Nutzung,Fokus Innovation,Fokus Empirischer Lernerfolg,Softwareklasse,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,X-Koordinate,Y-Koordinate
12,2014,109,1,1,1,0,0,-1,1,-1,0,NaN,NaN,NaN,NaN,1.0,-1.0
44,2015,107,1,1,1,-1,-1,-1,1,1,0,NaN,NaN,NaN,NaN,0.0,-1.0
62,2016,35,1,1,1,-1,1,-1,1,1,0,NaN,NaN,NaN,NaN,0.0,-1.0
75,2016,179,1,1,1,1,1,-1,1,-1,0,NaN,NaN,NaN,NaN,1.0,0.0


### Paper titles & URLs:


- **Innovation: 2014 (p. 109-120): Raphael Zender, Julius Höfler, Patrick Wolfien, Ulrike Lucke; ScratchDrone – Systematische Programmierung von Flugdrohnen für den Informatikunterricht;  https://dl.gi.de/items/c3562874-f63b-4058-b973-df7479c746b8**

- Innovation & Lernerfolg: 2015 (p. 107-118): Simone Kirst, Dietmar Zoerner, Jan Schütze, Ulrike Lucke, Isabel Dziobek; Zirkus Empathico Eine mobile Applikation zum Training sozioemotionaler
Kompetenzen bei Kindern im Autismus-Spektrum; https://dl.gi.de/items/2daa2efc-4cdc-4fbd-a634-17637ca0b804

- Innovation & Lernerfolg: 2016 (p. 35-46): Wiebke Köhlmann, Marlene Karlapp;  Adaption und Evaluation eines virtuellen Klassenzimmers für Blinde; https://dl.gi.de/items/ddf29774-bc4d-4ac2-822b-0c2687250bc7

- Innovation: 2016 (p. 179-190): Fabian Lutze, Raphael Zender, Ulrike Lucke; HardDrive Exchange – Eine VR-Lernanwendung zur Durchführung von Festplattenwechseln in Speichersystemen; https://dl.gi.de/items/711808d2-d303-4380-83f9-b0c19b73b14c 

In [8]:
annotations_scratchdrone_paper = {
    "Paper": ["Scratchdrone", "Scratchdrone", "Scratchdrone", "Scratchdrone", "Scratchdrone"],
    "Annotator": ["Referenzpaper", "Claude", "ChatGPT", "Gemini", "Qwen3"],
    "Evaluation beschrieben": [1, 1, 1, 1, 1],
    "Einzelne Software": [1, 1, 1, 1, 1],
    "Lehr-Lern-Setting": [1, 1, 1, 1, 1],
    "Fokus Medialität/Prozess": [0, 0, -1, 0, 0],
    "Fokus Akzeptanz": [0, 1, 1, 1, 1],
    "Fokus Innovation": [1, 1, 1, 1, 1],
    "Fokus Empirischer Lernerfolg": [-1, 0, -1, 0, 1],
}

df_scratchdrone_paper = pd.DataFrame(annotations_scratchdrone_paper)

df_scratchdrone_paper

,Paper,Annotator,Evaluation beschrieben,Einzelne Software,Lehr-Lern-Setting,Fokus Medialität/Prozess,Fokus Akzeptanz,Fokus Innovation,Fokus Empirischer Lernerfolg
0,Scratchdrone,Referenzpaper,1,1,1,0,0,1,-1
1,Scratchdrone,Claude,1,1,1,0,1,1,0
2,Scratchdrone,ChatGPT,1,1,1,-1,1,1,-1
3,Scratchdrone,Gemini,1,1,1,0,1,1,0
4,Scratchdrone,Qwen3,1,1,1,0,1,1,1


#### 1.3.1.2) Lernerfolg Paper

In [9]:
df_subset_lernerfolg

,DelFI Band,Beitragstitelseite,Evaluation durchgeführt,Einzelne Software,Lehr-Lernsetting,Fokus Medialität,Fokus Akzeptanz,Fokus Normalität der Nutzung,Fokus Innovation,Fokus Empirischer Lernerfolg,Softwareklasse,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,X-Koordinate,Y-Koordinate
22,2014,193,1,1,1,-1,1,-1,-1,1,0,NaN,NaN,NaN,NaN,-1.0,0.0
28,2014,259,1,1,1,0,-1,-1,-1,1,0,NaN,NaN,NaN,NaN,-1.0,0.5
42,2015,81,1,1,1,-1,1,-1,0,1,0,NaN,NaN,NaN,NaN,-1.0,-0.5
44,2015,107,1,1,1,-1,-1,-1,1,1,0,NaN,NaN,NaN,NaN,0.0,-1.0
62,2016,35,1,1,1,-1,1,-1,1,1,0,NaN,NaN,NaN,NaN,0.0,-1.0
64,2016,59,1,1,1,-1,1,-1,0,1,-1,NaN,NaN,NaN,NaN,-1.0,-0.5
66,2016,83,1,1,1,1,1,1,0,1,0,NaN,NaN,NaN,NaN,-1.0,1.0
67,2016,93,1,1,1,1,1,1,0,1,0,NaN,NaN,NaN,NaN,-1.0,1.0
76,2016,192,1,1,1,-1,1,-1,-1,1,0,NaN,NaN,NaN,NaN,-1.0,0.0
80,2016,233,1,1,1,-1,-1,-1,-1,1,0,NaN,NaN,NaN,NaN,-1.0,0.0


In [10]:
## draw a random sample of n = 2 from lernerfolg 

sample_lernerfolg = df_subset_lernerfolg.sample(n = 2, random_state=42)
sample_lernerfolg

,DelFI Band,Beitragstitelseite,Evaluation durchgeführt,Einzelne Software,Lehr-Lernsetting,Fokus Medialität,Fokus Akzeptanz,Fokus Normalität der Nutzung,Fokus Innovation,Fokus Empirischer Lernerfolg,Softwareklasse,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,X-Koordinate,Y-Koordinate
76,2016,192,1,1,1,-1,1,-1,-1,1,0,NaN,NaN,NaN,NaN,-1.0,0.0
28,2014,259,1,1,1,0,-1,-1,-1,1,0,NaN,NaN,NaN,NaN,-1.0,0.5


### Paper titles & URLS:

- 2016 (p. 191-202): Michael Eichhorn; Elektronische Abstimmungssysteme in der Hochschullehre – Empirische Untersuchung zu ersten Erfahrungen mit dem Audience Response System eduVote; https://dl.gi.de/items/dc694623-be6a-4f9a-883c-8b5122368bf3

- **2014 (p. 259-264): Marianus Ifland, Michael Jedich, Christian Schneider, Frank Puppe; ÜPS – Ein autorenfreundliches Trainingssystem für SQLAnfragen; https://dl.gi.de/items/7ed65e4d-a65d-4b08-ab4f-f284eae42ec7**


In [11]:
annotations_üps_paper = {
    "Paper": ["ÜPS", "ÜPS", "ÜPS", "ÜPS", "ÜPS"],
    "Annotator": ["Referenzpaper", "Claude", "ChatGPT", "Gemini", "Qwen3"],
    "Evaluation beschrieben": [1, 1, 1, 1, 1],
    "Einzelne Software": [1, 1, 1, 1, 1],
    "Lehr-Lern-Setting": [1, 1, 1, 1, 1],
    "Fokus Medialität/Prozess": [0, 1, 1, 0, 1],
    "Fokus Akzeptanz": [-1, 1, 1, 1, 1],
    "Fokus Innovation": [-1, 0, -1, 0, 0],
    "Fokus Empirischer Lernerfolg": [1, 0, 1, 1, 1],
}

df_üps_paper = pd.DataFrame(annotations_üps_paper)

df_üps_paper

,Paper,Annotator,Evaluation beschrieben,Einzelne Software,Lehr-Lern-Setting,Fokus Medialität/Prozess,Fokus Akzeptanz,Fokus Innovation,Fokus Empirischer Lernerfolg
0,ÜPS,Referenzpaper,1,1,1,0,-1,-1,1
1,ÜPS,Claude,1,1,1,1,1,0,0
2,ÜPS,ChatGPT,1,1,1,1,1,-1,1
3,ÜPS,Gemini,1,1,1,0,1,0,1
4,ÜPS,Qwen3,1,1,1,1,1,0,1


#### 1.3.1.3) Innovation & Lernerfolg Paper

In [12]:
df_subset_innovation_lernerfolg

,DelFI Band,Beitragstitelseite,Evaluation durchgeführt,Einzelne Software,Lehr-Lernsetting,Fokus Medialität,Fokus Akzeptanz,Fokus Normalität der Nutzung,Fokus Innovation,Fokus Empirischer Lernerfolg,Softwareklasse,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,X-Koordinate,Y-Koordinate
44,2015,107,1,1,1,-1,-1,-1,1,1,0,NaN,NaN,NaN,NaN,0.0,-1.0
62,2016,35,1,1,1,-1,1,-1,1,1,0,NaN,NaN,NaN,NaN,0.0,-1.0


### Paper titles & URLs:

- **Innovation & Lernerfolg: 2015 (p. 107-118): Simone Kirst, Dietmar Zoerner, Jan Schütze, Ulrike Lucke, Isabel Dziobek; Zirkus Empathico Eine mobile Applikation zum Training sozioemotionaler
Kompetenzen bei Kindern im Autismus-Spektrum; https://dl.gi.de/items/2daa2efc-4cdc-4fbd-a634-17637ca0b804**

- Innovation & Lernerfolg: 2016 (p. 35-46): Wiebke Köhlmann, Marlene Karlapp;  Adaption und Evaluation eines virtuellen Klassenzimmers für Blinde; https://dl.gi.de/items/ddf29774-bc4d-4ac2-822b-0c2687250bc7

In [13]:
annotations_zirkus_paper = {
    "Paper": ["Zirkus", "Zirkus", "Zirkus", "Zirkus", "Zirkus"],
    "Annotator": ["Referenzpaper", "Claude", "ChatGPT", "Gemini", "Qwen3"],
    "Evaluation beschrieben": [1, 1, 1, 1, 1],
    "Einzelne Software": [1, 1, 1, 1, 1],
    "Lehr-Lern-Setting": [1, 1, 1, 1, 1],
    "Fokus Medialität/Prozess": [-1, 0, 0, 0, 0],
    "Fokus Akzeptanz": [-1, 1, 1, 1, 1],
    "Fokus Innovation": [1, 1, 1, 1, 0],
    "Fokus Empirischer Lernerfolg": [1, 1, 1, 1, 1],
}

df_zirkus_paper = pd.DataFrame(annotations_zirkus_paper)

df_zirkus_paper

,Paper,Annotator,Evaluation beschrieben,Einzelne Software,Lehr-Lern-Setting,Fokus Medialität/Prozess,Fokus Akzeptanz,Fokus Innovation,Fokus Empirischer Lernerfolg
0,Zirkus,Referenzpaper,1,1,1,-1,-1,1,1
1,Zirkus,Claude,1,1,1,0,1,1,1
2,Zirkus,ChatGPT,1,1,1,0,1,1,1
3,Zirkus,Gemini,1,1,1,0,1,1,1
4,Zirkus,Qwen3,1,1,1,0,1,0,1


In [14]:
# Combine all into one DataFrame
df_all_papers = pd.concat([df_scratchdrone_paper, df_üps_paper, df_zirkus_paper], ignore_index=True)
df_all_papers

,Paper,Annotator,Evaluation beschrieben,Einzelne Software,Lehr-Lern-Setting,Fokus Medialität/Prozess,Fokus Akzeptanz,Fokus Innovation,Fokus Empirischer Lernerfolg
0,Scratchdrone,Referenzpaper,1,1,1,0,0,1,-1
1,Scratchdrone,Claude,1,1,1,0,1,1,0
2,Scratchdrone,ChatGPT,1,1,1,-1,1,1,-1
3,Scratchdrone,Gemini,1,1,1,0,1,1,0
4,Scratchdrone,Qwen3,1,1,1,0,1,1,1
5,ÜPS,Referenzpaper,1,1,1,0,-1,-1,1
6,ÜPS,Claude,1,1,1,1,1,0,0
7,ÜPS,ChatGPT,1,1,1,1,1,-1,1
8,ÜPS,Gemini,1,1,1,0,1,0,1
9,ÜPS,Qwen3,1,1,1,1,1,0,1


#### 1.3.3.4) Calculate Inter-Coder Reliability Metrics

In [15]:

# Now reshape from “wide by variable” to long format
df_long = df_all_papers.melt(
    id_vars=["Paper", "Annotator"],
    value_vars=[
        "Evaluation beschrieben",
        "Einzelne Software",
        "Lehr-Lern-Setting",
        "Fokus Medialität/Prozess",
        "Fokus Akzeptanz",
        "Fokus Innovation",
        "Fokus Empirischer Lernerfolg"
    ],
    var_name="Variable",
    value_name="Rating"
)

# Example of df_long head
#print(df_long.head(10))


df_wide = df_long.pivot_table(
    index=["Paper", "Variable"],
    columns="Annotator",
    values="Rating"
).reset_index()

#print(df_wide.head())


def compute_all_pairwise_kappas(df_wide, annotator_cols, weights_list=[None, "linear"]):
    """
    Compute pairwise Cohen's kappa (unweighted or weighted) for all annotator pairs.
    
    Params
    ------
    df_wide : pandas.DataFrame
        DataFrame in “wide” format — each row is one item (e.g. paper × variable),
        and each annotator has a separate column in annotator_cols.
    annotator_cols : list of str
        Column names of annotators (e.g. ["Referenzpaper","Claude","ChatGPT",...]).
    weights_list : list
        List of `weights` parameter values to pass to cohen_kappa_score.
        Example: [None, "linear"]
    
    Returns
    -------
    pandas.DataFrame with columns:
      - annotator1
      - annotator2
      - kappa_{unweighted}  (if weights_list includes None)
      - kappa_{linear}      (if weights_list includes "linear")
    """
    results = []
    for a1, a2 in combinations(annotator_cols, 2):
        row = {"annotator1": a1, "annotator2": a2}
        y1 = df_wide[a1]
        y2 = df_wide[a2]
        
        for w in weights_list:
            try:
                k = cohen_kappa_score(y1, y2, weights=w)
            except Exception:
                k = float("nan")
            key = "kappa_unweighted" if w is None else f"kappa_{w}"
            row[key] = k
        results.append(row)
    
    return pd.DataFrame(results)


# Example usage:

# Assuming df_wide is your “long→pivoted” DataFrame such that
# columns are annotators and rows are (paper × variable).
# E.g.:
# df_wide.columns = ["Paper", "Variable", "Referenzpaper", "Claude", "ChatGPT", "Gemini", "Qwen3"]

annotators = ["Referenzpaper", "Claude", "ChatGPT", "Gemini", "Qwen3"]
df_pairwise = compute_all_pairwise_kappas(df_wide, annotators, weights_list=[None, "linear"])

# To sort ascending by e.g. unweighted kappa:
df_pairwise = df_pairwise.sort_values(by="kappa_unweighted", ascending=False)

print(df_pairwise)


      annotator1 annotator2  kappa_unweighted  kappa_linear
5         Claude     Gemini          0.737500      0.737500
6         Claude      Qwen3          0.577181      0.577181
9         Gemini      Qwen3          0.577181      0.577181
4         Claude    ChatGPT          0.487805      0.603774
7        ChatGPT     Gemini          0.487805      0.603774
8        ChatGPT      Qwen3          0.432432      0.472362
2  Referenzpaper     Gemini          0.422018      0.436242
1  Referenzpaper    ChatGPT          0.376238      0.484663
0  Referenzpaper     Claude          0.229358      0.295302
3  Referenzpaper      Qwen3          0.192308      0.211604


#### Interpretation of Cohen's Kappa scores e.g., [Source](https://pmc.ncbi.nlm.nih.gov/articles/PMC3900052/): 

Level of Agreement

- 0 - 0.20 = None 
- 0.21 - 0.39 = Minimal
- 0.40 - 0.59 = Weak
- 0.60 - 0.79 = Moderate
- 0.80 - 0.90 = Strong
- more than 0.9 = Almost Perfect


In [16]:
# Analyse, wie viele LLMs (& Referenzpaper) mit dem Prompt Nr.1 bei wie vielen Papern "Fokus Medialität/Prozess" = 1 
# & "Fokus empirischer Lernerfolg" = 1 gelabelt haben (sollten wenige oder gar keine sein)

df_all_papers.loc[(df_all_papers["Fokus Medialität/Prozess"] == 1) & (df_all_papers["Fokus Empirischer Lernerfolg"] == 1)]

,Paper,Annotator,Evaluation beschrieben,Einzelne Software,Lehr-Lern-Setting,Fokus Medialität/Prozess,Fokus Akzeptanz,Fokus Innovation,Fokus Empirischer Lernerfolg
7,ÜPS,ChatGPT,1,1,1,1,1,-1,1
9,ÜPS,Qwen3,1,1,1,1,1,0,1


### 1.3.2) Data from DeLFI Papers after 2016 (not part of the [Reference Paper](https://dl.gi.de/server/api/core/bitstreams/55b17878-2f79-40ce-b574-313db7cfaf11/content))


In [17]:
## 1) 2019 (lni297)

# Paper = Leseflüssigkeitstraining mit Amazon Alexa


annotations_alexa_paper = {
    "Paper": ["Alexa", "Alexa", "Alexa"],
    "Annotator": ["ChatGPT", "Gemini", "Qwen3"],
    "Evaluation beschrieben": [1,1,1],
    "Einzelne Software": [1,1,1],
    "Lehr-Lern-Setting": [1,1,1],
    "Fokus Medialität/Prozess": [0,0,0],
    "Fokus Akzeptanz": [1,1,1],
    "Fokus Innovation": [0,1,0],
    "Fokus Empirischer Lernerfolg": [0,0,0],
}

df_alexa_paper = pd.DataFrame(annotations_alexa_paper)

df_alexa_paper


,Paper,Annotator,Evaluation beschrieben,Einzelne Software,Lehr-Lern-Setting,Fokus Medialität/Prozess,Fokus Akzeptanz,Fokus Innovation,Fokus Empirischer Lernerfolg
0,Alexa,ChatGPT,1,1,1,0,1,0,0
1,Alexa,Gemini,1,1,1,0,1,1,0
2,Alexa,Qwen3,1,1,1,0,1,0,0


In [18]:
## 2) 2024 (lni356)

# Paper: Alexander-Gantikow: KI-basierte Analyse von E-Portfolios 


annotations_portfolios_paper = {
    "Paper": ["E-Portfolios", "E-Portfolios", "E-Portfolios"],
    "Annotator": ["ChatGPT", "Gemini", "Qwen3"],
    "Evaluation beschrieben": [1,1,1],
    "Einzelne Software": [1,1,1],
    "Lehr-Lern-Setting": [1,1,1],
    "Fokus Medialität/Prozess": [1,1,1],
    "Fokus Akzeptanz": [1,1,1],
    "Fokus Innovation": [0,1,0],
    "Fokus Empirischer Lernerfolg": [0,0,0],
}

df_portfolios_paper = pd.DataFrame(annotations_portfolios_paper)

df_portfolios_paper


,Paper,Annotator,Evaluation beschrieben,Einzelne Software,Lehr-Lern-Setting,Fokus Medialität/Prozess,Fokus Akzeptanz,Fokus Innovation,Fokus Empirischer Lernerfolg
0,E-Portfolios,ChatGPT,1,1,1,1,1,0,0
1,E-Portfolios,Gemini,1,1,1,1,1,1,0
2,E-Portfolios,Qwen3,1,1,1,1,1,0,0


In [19]:
# Combine all into one DataFrame
df_all_papers_after_2017 = pd.concat([df_alexa_paper, df_portfolios_paper], ignore_index=True)
df_all_papers_after_2017

,Paper,Annotator,Evaluation beschrieben,Einzelne Software,Lehr-Lern-Setting,Fokus Medialität/Prozess,Fokus Akzeptanz,Fokus Innovation,Fokus Empirischer Lernerfolg
0,Alexa,ChatGPT,1,1,1,0,1,0,0
1,Alexa,Gemini,1,1,1,0,1,1,0
2,Alexa,Qwen3,1,1,1,0,1,0,0
3,E-Portfolios,ChatGPT,1,1,1,1,1,0,0
4,E-Portfolios,Gemini,1,1,1,1,1,1,0
5,E-Portfolios,Qwen3,1,1,1,1,1,0,0


In [20]:
# Now reshape from “wide by variable” to long format
df_long = df_all_papers_after_2017.melt(
    id_vars=["Paper", "Annotator"],
    value_vars=[
        "Evaluation beschrieben",
        "Einzelne Software",
        "Lehr-Lern-Setting",
        "Fokus Medialität/Prozess",
        "Fokus Akzeptanz",
        "Fokus Innovation",
        "Fokus Empirischer Lernerfolg"
    ],
    var_name="Variable",
    value_name="Rating"
)

# Example of df_long head
#print(df_long.head(10))


df_wide = df_long.pivot_table(
    index=["Paper", "Variable"],
    columns="Annotator",
    values="Rating"
).reset_index()



annotators = ["ChatGPT", "Gemini", "Qwen3"]
df_pairwise = compute_all_pairwise_kappas(df_wide, annotators, weights_list=[None, "linear"])

# To sort ascending by e.g. unweighted kappa:
df_pairwise = df_pairwise.sort_values(by="kappa_unweighted", ascending=False)

print(df_pairwise)


  annotator1 annotator2  kappa_unweighted  kappa_linear
1    ChatGPT      Qwen3          1.000000      1.000000
0    ChatGPT     Gemini          0.658537      0.658537
2     Gemini      Qwen3          0.658537      0.658537


##  Offene Fragen

- Ist mit "Fokus Medialität" Paradigma "Effizienz" gemeint?
- Was ist der Unterschied zwischen "Fokus Akzeptanz" und "Fokus Normalität der Nutzung"?
- Wie viel Sinn macht die implizite Codierung ("0") der Paradigmen?

## Beobachtungen


- größte Abweichungen zw. Human & LLM annotations -> Kategorie "Akzeptanz" (alle 3 Paper 2014-2016)

- Modelle verstehen unterschiedliche Dinge unter "Prozess/Effizienz"
	- Evaluationsergebnisse vs. "Die Effizienz des _Prozesses_ (z.B. Zeitersparnis für Autoren oder Studierende) wird nicht explizit gemessen oder als Hauptziel genannt."
	- *vllt noch besser definieren in prompt*  
- **Prompt template mit Hilfe von Julian optimieren**
	- 0-Shot vs ICL
	- including LLM explanations or not?
		- would need LLM-as-a-judge to evaluate
- **How to verify annotations of LLMs?**
	- using multiple LLMs -> cross-consistency checks 
		- lowest ICR -> human re-annotation
		- possible in MEGAnno+ pipeline?
	- using model logits 
		- vgl. MEGAnno+ -> confidence-score (OpenAI API provides that)
	- calculating consistency over prompt perturbations/ prompt the LLM multiple times with different prompt format to measure its consistency 
		- Xuezhi Wang, Jason Wei, Dale Schuurmans, Quoc V Le, Ed H. Chi, Sharan Narang, Aakanksha Chowdhery, and Denny Zhou. 2023. [Self-consistency improves chain of thought reasoning in language models](https://openreview.net/forum?id=1PL1NIMMrw). In The Eleventh International Conference on Learning Representations.
	- verbalized confidence prompting 
		- *problem of over-confidence*
	- different verifier models (vgl. Human-LLM Collaborative Annotation Through Effective Verification of LLM Labels)
		- trained verifier (needs gold labels)
		- baseline verifiers
			- logits
			- entropies

## Ideen: 

- Labels/Paradigmen (2nd layer) explizit definieren: 
	- Effizienz, Lernerfolg, Innovation, Akzeptanz
	- *ist aber eigentlich schon definiert in Referenzpaper, halt als Frage*
    - Vgl. Operationalisierung!

# 2) Experiments Nr. 2:


## Prompt Template Nr. 2: 

### Notizen zu Prompt Template Variation 

- Akzeptanzorientiertes Paradigma: "Wurde die Akzeptanz der Software in der Breite, die Usability oder die Normalität der Nutzung beleuchtet? (-1,0,1)"
    - Wording ändern: 
        - "Akzeptanz" herausnehmen und lieber Begriffe wie "gesellschaftliche Entwicklung/Trend" verwenden 
        - "Usability" als Begriff aus der Definition streichen -> gehört eher zu dem Bauen Paradigma, i.e. Innovativ-künstlerische Paradigma

    - Bedeutung des Paradigmas:
        - Software wurde nicht für einen bestimmten Zweck/Forschung gebaut
        - keine selbst gebaute Software 


-  Innovativ-künstlerische Paradigma: "Wurde die Neuigkeit oder Ästhetik der verwendeten Technologien/Konzepte beleuchtet? (-1,0,1)"

    - selbst gebaute Software
    - Bauen der Software steht im Vordergrund, muss nicht unbedingt "innovativ/neu/ästhetisch" in dem Software-Kapitel einer DeLFI-Publikation stehen
        - evtl. hier auch semantisches Problem der LLMs mit bisheriger Definition: fokussieren zu sehr auf Innovation/Neuigkeit/Ästhetik? 


- Prozess orientiertes Paradigma & Ergebnis orientiertes Paradigma:

    - als mutually exclusive/gegensätzliche Paradigmen im Prompt beschreiben (als relatives Wahrscheinlichkeitskonzept einführen) und testen


- Generell: neue Begriffe zu Reserach Software Engineering austesten, zB: "Forschungssoftware", "Research software" 


- Methodisch/methodologisch: immer erst einen Teil des Prompt Templates ändern (und nicht alle Änderungen auf einmal) und anschließend testen und die Annotationen miteinander vergleichen

    - [Paper: self-consistency prompting](https://openreview.net/pdf?id=1PL1NIMMrw)
